# Benchmark for Database Loading

Sample : HITEMP H$_2$O `01_1000-1150_HITEMP2010` (4.9 billion lines, 700 MB .h5 cache file)

### Tests

1. [#HDF5-is-must-faster-than-SQL-databases](#HDF5-is-must-faster-than-SQL-databases)
2. [#Improving-HDF5-performances](#Improving-HDF5-performances)
   - [#Trying-different-compression](#Trying-different-compression)
   - [#Test-reading-fewer-columns](#Test-reading-fewer-columns)
   - [#Adjusting-dtypes-and-precision](#Adjusting-dtypes-and-precision)


### Helpful references

- https://stackoverflow.com/questions/16628329/hdf5-concurrency-compression-i-o-performance
- https://www.dataquest.io/blog/python-pandas-databases/
- https://www.researchgate.net/publication/282986584_Efficient_Management_of_Big_Datasets_Using_HDF_and_SQLite_A_Comparative_Study_Based_on_Building_Simulation_Data
- https://colab.research.google.com/github/kastnerkyle/kastnerkyle.github.io/blob/master/posts/using-pytables-for-larger-than-ram-data-processing/using-pytables-for-larger-than-ram-data-processing.ipynb#scrollTo=GUviW4aJ54Tc


# Initialize : 

In [1]:
import pandas as pd

In [19]:
datafile = "01_1000-1150_HITEMP2010"
df = pd.read_hdf(datafile+".h5")       # Assuming .H5 already created (fixed-form : RADIS default)

In [3]:
print(len(df), "lines")
# number of isotopes : 
print("Isotopes : ", df.iso.unique())

4960367 lines
Isotopes :  [1 2 4 3]


In [4]:
import os
if not os.path.exists(datafile+".sql"):
    # Generating to the database
    import sqlite3
    conn = sqlite3.connect(datafile+".sql")

    # Setting up the database
    df.to_sql("HITEMP_table", conn)

    conn.close()

# HDF5 is must faster than SQL-databases

(at least the sqlite3 implementation)

## Test Reading everything 


### HDF5

In [5]:
%timeit -r 3 -n 1 pd.read_hdf(datafile+".h5")

3.46 s ± 58.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### SQL

In [6]:
# Connecting to the database
import sqlite3
conn = sqlite3.connect(datafile+".sql")
%timeit -r 3 -n 1 pd.read_sql("select * from HITEMP_table ", conn)

conn.close()

1min 16s ± 1.2 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [7]:
df

,id,iso,wav,int,A,airbrd,selbrd,El,Tdpair,Pshft,...,Kcu,jl,Kal,Kcl,v1u,v2u,v3u,v1l,v2l,v3l
0,1,1,1000.0001,7.598000e-75,0.016460,0.0560,0.315,25339.8633,0.49,0.0,...,-1,9,-2,-2,-2,-2,-2,-2,-2,-2
1,1,1,1000.0001,7.260000e-79,0.918300,0.0075,0.150,28248.8438,0.41,0.0,...,-1,21,-2,-2,-2,-2,-2,-2,-2,-2
2,1,1,1000.0001,1.642000e-68,0.177000,0.0075,0.131,23005.1113,0.41,0.0,...,-2,23,-1,-1,-2,-2,-2,-2,-2,-2
3,1,1,1000.0001,8.974000e-69,15.320000,0.0166,0.114,24080.7578,0.50,0.0,...,-2,25,-1,-1,-2,-2,-2,-2,-2,-2
4,1,1,1000.0001,1.380000e-77,0.040330,0.0501,0.298,27069.1562,0.45,0.0,...,-3,10,-4,-4,-2,-2,-2,-2,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960362,1,1,1149.9999,9.634000e-79,0.048040,0.0241,0.208,27463.1426,0.38,0.0,...,-2,16,-1,-1,-2,-2,-2,-2,-2,-2
4960363,1,1,1149.9999,2.623000e-78,0.009012,0.0089,0.047,27318.4102,0.50,0.0,...,-3,39,-4,-4,-2,-2,-2,-2,-2,-2
4960364,1,1,1149.9999,1.260000e-68,0.050910,0.0126,0.086,23028.3086,0.50,0.0,...,-4,29,-3,-3,-2,-2,-2,-2,-2,-2
4960365,1,1,1150.0000,2.205000e-80,0.068240,0.0075,0.131,28393.7812,0.41,0.0,...,-1,23,-2,-2,-2,-2,-2,-2,-2,-2


## Test using SQL features (partial loading)

For instance only load isotope=2

In [8]:
(df.iso==2).sum()

42

In [9]:
# Must store in Table format (not Fixed Format)
df.to_hdf(datafile+"-table.h5", key="HITEMP_table", format='table', data_columns=['iso'])

In [11]:
pd.read_hdf(datafile+"-table.h5", where = 'iso in [2]')
df

,id,iso,wav,int,A,airbrd,selbrd,El,Tdpair,Pshft,...,Kcu,jl,Kal,Kcl,v1u,v2u,v3u,v1l,v2l,v3l
0,1,1,1000.0001,7.598000e-75,0.016460,0.0560,0.315,25339.8633,0.49,0.0,...,-1,9,-2,-2,-2,-2,-2,-2,-2,-2
1,1,1,1000.0001,7.260000e-79,0.918300,0.0075,0.150,28248.8438,0.41,0.0,...,-1,21,-2,-2,-2,-2,-2,-2,-2,-2
2,1,1,1000.0001,1.642000e-68,0.177000,0.0075,0.131,23005.1113,0.41,0.0,...,-2,23,-1,-1,-2,-2,-2,-2,-2,-2
3,1,1,1000.0001,8.974000e-69,15.320000,0.0166,0.114,24080.7578,0.50,0.0,...,-2,25,-1,-1,-2,-2,-2,-2,-2,-2
4,1,1,1000.0001,1.380000e-77,0.040330,0.0501,0.298,27069.1562,0.45,0.0,...,-3,10,-4,-4,-2,-2,-2,-2,-2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960362,1,1,1149.9999,9.634000e-79,0.048040,0.0241,0.208,27463.1426,0.38,0.0,...,-2,16,-1,-1,-2,-2,-2,-2,-2,-2
4960363,1,1,1149.9999,2.623000e-78,0.009012,0.0089,0.047,27318.4102,0.50,0.0,...,-3,39,-4,-4,-2,-2,-2,-2,-2,-2
4960364,1,1,1149.9999,1.260000e-68,0.050910,0.0126,0.086,23028.3086,0.50,0.0,...,-4,29,-3,-3,-2,-2,-2,-2,-2,-2
4960365,1,1,1150.0000,2.205000e-80,0.068240,0.0075,0.131,28393.7812,0.41,0.0,...,-1,23,-2,-2,-2,-2,-2,-2,-2,-2


In [12]:
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-table.h5", where ='iso in [2]')

39.2 ms ± 2.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


Now with SQL

In [13]:
# Connecting to the database
import sqlite3
conn = sqlite3.connect(datafile+".sql")
%timeit -r 3 -n 1 pd.read_sql("select * from HITEMP_table where iso=2", conn)

1.26 s ± 9.43 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Conclusions

SQLlite is very slow compared to HDF5 (both read & write is ~ 1 order of magnitude slower)


Note:  for HDF5 to be able to use `where` similar to SQL, a table (and not fixed-form) format must be used. This ends up taking ~50% more space on disk in my case : 

In [41]:
from os.path import getsize
print("{0:.0f} MB".format(getsize(datafile+".h5")*1e-6))
print("{0:.0f} MB".format(getsize(datafile+"-table.h5")*1e-6))

724 MB
1034 MB


# Improving HDF5 performances

## Trying different compression

References : 
- http://www.pytables.org/usersguide/optimization.html?highlight=optimization#compressionissues



In [42]:
df.to_hdf(datafile+"-zlib1.h5", key="HITEMP_table", format="table", complib='zlib', complevel=1)
df.to_hdf(datafile+"-zlib9.h5", key="HITEMP_table", format="table", complib='zlib', complevel=9)
df.to_hdf(datafile+"-blosc1.h5", key="HITEMP_table", format="table", complib='blosc', complevel=1)
df.to_hdf(datafile+"-blosc9.h5", key="HITEMP_table", format="table", complib='blosc', complevel=9)
df.to_hdf(datafile+"-lzo1.h5", key="HITEMP_table", format="table", complib='lzo', complevel=1)
df.to_hdf(datafile+"-lzo9.h5", key="HITEMP_table", format="table", complib='lzo', complevel=9)

Read velocity. Conclusion: blosc almost as fast as no-compression. LZO decently fast, but Zlib is twice as slow. 

In [43]:
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-zlib1.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-zlib9.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-blosc1.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-blosc9.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-lzo1.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-lzo9.h5")

8 s ± 190 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
7.89 s ± 36.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.2 s ± 173 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.06 s ± 25.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
5.42 s ± 17.7 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
5.44 s ± 31.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [44]:
# as a reference, no compression : 
%timeit -r 3 -n 1 pd.read_hdf(datafile+".h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-table.h5")

3.47 s ± 45.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.73 s ± 70.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


Size of disk (and compression) : 
- Zlib achieves the best compression, LZO is 2nd and Blosc9 is about equivalent, Blosc1 is the worst. 

In [46]:
from os.path import getsize
ref = getsize(datafile+"-table.h5")
print("Reference:", "{0:.0f} MB".format(ref*1e-6))
for complib in ["zlib1", "zlib9", "blosc1", "blosc9", "lzo1", "lzo9"]:
    print(complib,":","{0:.0f} MB ({1:.0f}% compression)".format(getsize(datafile+"-"+complib+".h5")*1e-6, 100-getsize(datafile+"-"+complib+".h5")/ref*100))

Reference: 1034 MB
zlib1 : 273 MB (74% compression)
zlib9 : 239 MB (77% compression)
blosc1 : 467 MB (55% compression)
blosc9 : 325 MB (69% compression)
lzo1 : 307 MB (70% compression)
lzo9 : 307 MB (70% compression)


### Conclusion:

- Blosc9 looks like a good mix. Not too much reading overhead, and good compression ! 
- Note that it's faster to read the blosc9 array than to read the non-compressed Table array (only Fixed-Form array is faster, but that's not suitable for SQL-like look-ups!)

## Test reading fewer columns

In [47]:
df.columns

Index(['id', 'iso', 'wav', 'int', 'A', 'airbrd', 'selbrd', 'El', 'Tdpair',
       'Pshft', 'gp', 'gpp', 'ju', 'Kau', 'Kcu', 'jl', 'Kal', 'Kcl', 'v1u',
       'v2u', 'v3u', 'v1l', 'v2l', 'v3l'],
      dtype='object')

In [48]:
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-table.h5")
%timeit -r 3 -n 1 pd.read_hdf(datafile+"-table.h5", columns=['wav', 'int'])

4.7 s ± 65.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.86 s ± 7.94 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


So reading fewer columns doesn't seem to improve read velocity. Of course it reduces the memory usage of the dataframe created, so it's worth considering to reduce RADIS RAM usage: 

In [55]:
dft = pd.read_hdf(datafile+"-table.h5")
dft2 = pd.read_hdf(datafile+"-table.h5", columns=['wav', 'int'])
import sys
print("{0:.0f} MB".format(sys.getsizeof(dft)*1e-6))
print("{0:.0f} MB".format(sys.getsizeof(dft2)*1e-6))

992 MB
119 MB


### Conclusions

- Does not improve velocity
- But it does reduce instantaneous RAM usage

## Adjusting dtypes and precision

Remember that HITRAN .par files have only 5-6 significant digits.

In [57]:
df.dtypes

id          int64
iso         int64
wav       float64
int       float64
A         float64
airbrd    float64
selbrd    float64
El        float64
Tdpair    float64
Pshft     float64
gp        float64
gpp       float64
ju          int64
Kau         int64
Kcu         int64
jl          int64
Kal         int64
Kcl         int64
v1u         int64
v2u         int64
v3u         int64
v1l         int64
v2l         int64
v3l         int64
dtype: object

In [59]:
df.wav

0          1000.0001
1          1000.0001
2          1000.0001
3          1000.0001
4          1000.0001
             ...    
4960362    1149.9999
4960363    1149.9999
4960364    1149.9999
4960365    1150.0000
4960366    1150.0000
Name: wav, Length: 4960367, dtype: float64

In [67]:
df.iloc[0].wav==1000.0001

True

Reducing the precision to float32 can save some tens of MBs per column ! 

In [92]:
import sys
import numpy as np 
print("{0:.0f} MB".format(sys.getsizeof(df.wav)*1e-6))
print("{0:.0f} MB".format(sys.getsizeof(df.wav.astype(np.float32))*1e-6))

79 MB
60 MB


But it changes the values : 

In [77]:
df.wav.astype(np.float32)

0          1000.000122
1          1000.000122
2          1000.000122
3          1000.000122
4          1000.000122
              ...     
4960362    1149.999878
4960363    1149.999878
4960364    1149.999878
4960365    1150.000000
4960366    1150.000000
Name: wav, Length: 4960367, dtype: float32

Is it still accurate given within the HITRAN significant digits ? 

In [88]:
np.round(df.wav.astype(np.float32).astype(np.float64), decimals=4) == df.wav

0          True
1          True
2          True
3          True
4          True
           ... 
4960362    True
4960363    True
4960364    True
4960365    True
4960366    True
Name: wav, Length: 4960367, dtype: bool

For many lines, yes. But not all :  

In [93]:
(np.round(df.wav.astype(np.float32).astype(np.float64), decimals=4) == df.wav).all()

False

Actually only 84% of the line positions would match :

In [90]:
(np.round(df.wav.astype(np.float32).astype(np.float64), decimals=4) == df.wav).sum()/len(df.wav)*100

84.51735526826947

### Conclusion

- reducing the precision not considered at this point ! 

# TODO 

From the excellent [using-pytables-for-larger-than-ram-data-processing.ipynb](https://colab.research.google.com/github/kastnerkyle/kastnerkyle.github.io/blob/master/posts/using-pytables-for-larger-than-ram-data-processing/using-pytables-for-larger-than-ram-data-processing.ipynb#scrollTo=GUviW4aJ54Tc) : 

- [ ] use blosc9. See examples above.

- [x] adjust dtypes. Remember that HITRAN .par files have only 5-6 significant digits. See .Atom for dtypes ? See [#Adjusting-dtypes-and-precision](#Adjusting-dtypes-and-precision)


- [ ] Refactor RADIS code: only read line data as we need it. For instance, we need all energy levels to compute the populations first, but no need to read broadening coefficients before we enter the broadening step. 
  - This can be used to fetch the remaining data : https://stackoverflow.com/questions/30483754/pandas-get-specific-rows-from-hdf5-by-index
  - Check that it's actually faster. Does data reading decrease linearly with the number of columns?  [#Test-reading-fewer-columns](#Test-reading-fewer-columns) seems to suggest that no. 

Add Evib/Erot on the fly... How to add columns? 

- [x] `df.to_hdf(mode='a')` can only be used to append rows. https://stackoverflow.com/questions/48745033/how-to-insert-edit-a-column-in-an-existing-hdf5-datasetRaises .  We'll have to re-rewrite the whole HDF5 file. Which raises a few questions for RADIS : should we edit the *original* HDF5 file and therefore fetch Evib/Erot for all the spectral range, and all isotopes ? If we don't and create a file only for runtime use, will it be re-used? (a fitting procedure with different temperatures and a cutoff would require new lines all the time). An intermediate solution would be to generate this runtime file **without cutoff**, for the waverange & molecules/isotopes considered.  

Other ideas for RADIS : 

- [ ] Be able to generate .h5 file on-the-fly from a large .par / .bz2 file that does not fit into memory. 

- [ ] in Spectrum object, do not export Lines as a Pandas Dataframe but only the index of the index used in the 'original' .h5 database ? And let the Line-survey method connect the dots. 